In [31]:
import glob
import os
import pandas as pd
from datetime import datetime


# Read Maxim location for all the months

In [32]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*naamsestraat-35-maxim.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_maxim_df = pd.concat(df_list, ignore_index=True)
full_maxim_df.shape

del df_list

In [33]:
full_maxim_df.sort_values(by=['result_timestamp'],inplace=True)

In [34]:
#full_maxim_df['result_timestamp'].dt.month.unique()

In [35]:
full_maxim_df = full_maxim_df.resample('15min',closed = 'right', on='result_timestamp').mean()
full_maxim_df.reset_index(inplace=True)

In [77]:
full_maxim_df.to_parquet('Dataset/full_maxim_df.parquet')

In [78]:
test_maxim_df = pd.read_parquet('Dataset/full_maxim_df.parquet')
test_maxim_df

,result_timestamp,lamax,laeq,lceq,lcpeak
0,2022-02-28 08:15:00,61.2500,59.90625,70.75000,82.0625
1,2022-02-28 08:30:00,57.4375,55.59375,65.81250,77.5625
2,2022-02-28 08:45:00,65.1250,63.37500,70.56250,82.3750
3,2022-02-28 09:00:00,NaN,NaN,NaN,NaN
4,2022-02-28 09:15:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...
28537,2022-12-22 14:30:00,56.1875,54.46875,63.37500,75.6875
28538,2022-12-22 14:45:00,56.0000,53.90625,61.87500,74.5625
28539,2022-12-22 15:00:00,55.8125,53.90625,61.84375,74.6250
28540,2022-12-22 15:15:00,55.9375,53.84375,60.62500,74.0625


In [38]:
#train_maxim['result_timestamp'].dt.month.unique()

In [40]:
test_maxim_df['result_timestamp'].dt.month.unique()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [41]:
missing_df = test_maxim_df[test_maxim_df.isnull().any(axis=1)]

In [44]:
missing_df

,result_timestamp,lamax,laeq,lceq,lcpeak
3,2022-02-28 09:00:00,NaN,NaN,NaN,NaN
4,2022-02-28 09:15:00,NaN,NaN,NaN,NaN
5,2022-02-28 09:30:00,NaN,NaN,NaN,NaN
6,2022-02-28 09:45:00,NaN,NaN,NaN,NaN
7,2022-02-28 10:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...
28158,2022-12-18 15:45:00,NaN,NaN,NaN,NaN
28255,2022-12-19 16:00:00,NaN,NaN,NaN,NaN
28349,2022-12-20 15:30:00,NaN,NaN,NaN,NaN
28350,2022-12-20 15:45:00,NaN,NaN,NaN,NaN


In [45]:
missing_df['result_timestamp'].dt.month.unique()

array([ 2,  3,  6,  7, 10, 11, 12])

In [47]:
oct_df = test_maxim_df[test_maxim_df['result_timestamp'].dt.month == 10]

In [49]:
missing_oct_df = oct_df[oct_df.isnull().any(axis=1)]

In [50]:
missing_oct_df

,result_timestamp,lamax,laeq,lceq,lcpeak
22961,2022-10-25 12:30:00,NaN,NaN,NaN,NaN
22962,2022-10-25 12:45:00,NaN,NaN,NaN,NaN
22963,2022-10-25 13:00:00,NaN,NaN,NaN,NaN
22964,2022-10-25 13:15:00,NaN,NaN,NaN,NaN
22965,2022-10-25 13:30:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...
23578,2022-10-31 22:45:00,NaN,NaN,NaN,NaN
23579,2022-10-31 23:00:00,NaN,NaN,NaN,NaN
23580,2022-10-31 23:15:00,NaN,NaN,NaN,NaN
23581,2022-10-31 23:30:00,NaN,NaN,NaN,NaN


# read Xior location

In [51]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*xior.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_xior_df = pd.concat(df_list, ignore_index=True)
full_xior_df.shape


(16995383, 5)

In [53]:
full_xior_df.sort_values(by=['result_timestamp'],inplace=True)
full_xior_df = full_xior_df.resample('15min',closed = 'right', on='result_timestamp').mean()
full_xior_df.reset_index(inplace=True)
full_xior_df.to_parquet('Dataset/full_xior_df.parquet')

In [54]:
try_df = pd.read_parquet('Dataset/full_xior_df.parquet')

In [76]:
try_df.head(20)

,result_timestamp,lamax,laeq,lceq,lcpeak
0,2022-02-28 14:00:00,51.50000,49.93750,58.96875,70.5625
1,2022-02-28 14:15:00,55.12500,52.96875,66.06250,77.1250
2,2022-02-28 14:30:00,56.00000,52.81250,60.71875,72.8125
3,2022-02-28 14:45:00,55.87500,54.81250,65.18750,75.9375
4,2022-02-28 15:00:00,NaN,NaN,NaN,NaN
5,2022-02-28 15:15:00,56.18750,53.25000,65.62500,78.0625
6,2022-02-28 15:30:00,46.43750,45.06250,59.18750,69.9375
7,2022-02-28 15:45:00,55.59375,53.06250,61.18750,73.9375
8,2022-02-28 16:00:00,53.84375,51.25000,59.00000,74.0625
9,2022-02-28 16:15:00,57.06250,54.96875,61.75000,74.5000


# read Taste location

In [60]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*taste.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_taste_df = pd.concat(df_list, ignore_index=True)

In [67]:
full_taste_df.sort_values(by=['result_timestamp'],inplace=True)
full_taste_df = full_taste_df.resample('15min',closed = 'right', on='result_timestamp').mean()
full_taste_df.reset_index(inplace=True)
full_taste_df.to_parquet('Dataset/full_taste_df.parquet')

# read Kapel location

In [61]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*calvariekapel-ku-leuven.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_kapel_df = pd.concat(df_list, ignore_index=True)

In [68]:
full_kapel_df.sort_values(by=['result_timestamp'],inplace=True)
full_kapel_df = full_kapel_df.resample('15min',closed = 'right', on='result_timestamp').mean()
full_kapel_df.reset_index(inplace=True)
full_kapel_df.to_parquet('Dataset/full_kapel_df.parquet')

# read filosovia location

In [62]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*filosovia.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_filo_df = pd.concat(df_list, ignore_index=True)

In [69]:
full_filo_df.sort_values(by=['result_timestamp'],inplace=True)
full_filo_df = full_filo_df.resample('15min',closed = 'right', on='result_timestamp').mean()
full_filo_df.reset_index(inplace=True)
full_filo_df.to_parquet('Dataset/full_filo_df.parquet')

# read 81 location

In [63]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*81.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_81_df = pd.concat(df_list, ignore_index=True)

In [70]:
full_81_df.sort_values(by=['result_timestamp'],inplace=True)
full_81_df = full_81_df.resample('15min',closed = 'right', on='result_timestamp').mean()
full_81_df.reset_index(inplace=True)
full_81_df.to_parquet('Dataset/full_81_df.parquet')

# read kiosk location useless

In [64]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*stadspark.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_kiosk_df = pd.concat(df_list, ignore_index=True)

In [72]:
#full_kiosk_df.sort_values(by=['result_timestamp'],inplace=True)
#full_kiosk_df = full_kiosk_df.resample('15min',closed = 'right', on='result_timestamp').mean()
#full_kiosk_df.reset_index(inplace=True)
full_kiosk_df.to_parquet('Dataset/full_kiosk_df.parquet')

# read hof location

In [65]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*vrijthof.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_hof_df = pd.concat(df_list, ignore_index=True)

In [73]:
full_hof_df.sort_values(by=['result_timestamp'],inplace=True)
full_hof_df = full_hof_df.resample('15min',closed = 'right', on='result_timestamp').mean()
full_hof_df.reset_index(inplace=True)
full_hof_df.to_parquet('Dataset/full_hof_df.parquet')

# read his his-hears location

In [66]:
custom_date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
path = '/Users/siucheung/School/Modern Data Analytics/project/Full Data Set/'

csv_files = glob.glob(path + "/*/*hears.csv") 
df_list = (pd.read_csv(file, sep=';',
                       usecols=['result_timestamp','lamax','laeq','lceq','lcpeak'],
                       dtype={'lamax':'float16','laeq':'float16','lceq':'float16','lcpeak':'float16'},
                       parse_dates=['result_timestamp'],
                       date_parser=custom_date_parser) 
                       for file in csv_files)

full_hears_df = pd.concat(df_list, ignore_index=True)

In [74]:
full_hears_df.sort_values(by=['result_timestamp'],inplace=True)
full_hears_df = full_hears_df.resample('15min',closed = 'right', on='result_timestamp').mean()
full_hears_df.reset_index(inplace=True)
full_hears_df.to_parquet('Dataset/full_hears_df.parquet')